In this part of the assignment, we are performing some of the same sorts of tasks that you performed in the last assignment as well as a few new ones. In the code below, we focus on the new task, including 

* inserting new columns in a DataFrame using a function of one or more other column that you write yourself,
* using crosstab to determine the correlation between a column and another column or pair of columns,
* writing and reading DataFrames to an SQLite database.

In [1]:
import pandas as pd
import numpy as np
titanic_df = pd.read_csv('titanic.csv')
titanic_df.columns

Index(['PassengerID', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

We want to insert a new column which takes the values in 'Sex', i.e. 'male' and 'female' and creates a new column with corresponding values 1 and 2. Let us use pandas `insert` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.insert.html) to insert the new 'GCode' column right after the 'Sex' column. We use the numpy `where` method  (https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html) to do the "translation".

In [2]:
import numpy as np
titanic_df.insert(5,'GCode', np.where(titanic_df.Sex=='male', 1, 2))
titanic_df[['Sex','GCode']].head()

,Sex,GCode
0,male,1
1,female,2
2,female,2
3,female,2
4,male,1


Note that this really assign 1 to 'male' and 2 to everything else. Let us define a function that assigns 1 to 'male' and 2 to 'female'.

In [3]:
def encode(sex):
    if sex == 'male':
        return 1
    if sex == 'female':
        return 2

In [4]:
titanic_df.Sex.map(encode).head()

0    1
1    2
2    2
3    2
4    1
Name: Sex, dtype: int64

Let now use this function to create a new `GCode2` column. It should have the same values as `GCode1`.

In [5]:
titanic_df.insert(5,'GCode2',titanic_df.Sex.map(encode))
titanic_df[['Sex','GCode','GCode2']].head()

,Sex,GCode,GCode2
0,male,1,1
1,female,2,2
2,female,2,2
3,female,2,2
4,male,1,1


Finally, let do some cross tabulations to check if the columns were created correctly...

In [6]:
pd.crosstab(titanic_df.GCode,titanic_df.Sex)

Sex,female,male
GCode,,
1,0,577
2,314,0


In [7]:
pd.crosstab([titanic_df.GCode,titanic_df.GCode2],titanic_df.Sex)

,Sex,female,male
GCode,GCode2,,
1,1,0,577
2,2,314,0


In [8]:
pd.crosstab([titanic_df.GCode,titanic_df.GCode2],titanic_df.Sex=='male')

,Sex,False,True
GCode,GCode2,,
1,1,0,577
2,2,314,0


Keep that last "strange" one in mind when working on the crosstab in GrEx2 part 3 (a).

Finally, let us see how to read write to a local SQLite database. We explore two ways to do this:
* using SQAlchemy
* using the sqlite3 module

Check your Package Manager to make sure SQLAlchemy is aready installed. 

In [9]:
# Modules used to create xyz.db and connect to it
import sqlalchemy #installed package SQLAlchemy 1.1.6-1
from sqlalchemy import create_engine

# Alternative approach is to use sqlite3 module
import sqlite3 

In [10]:
ls # check to see that there is no xyz.db file since we have not created it yet...

GrEx 2 Progress Report 2 Examples.html
GrEx 2 Progress Report 2 Examples.ipynb
GrEx2 Progress Report 2 Examples v3.ipynb
titanic.csv
xyz.db


In [11]:
# create SQLAlchemy engine to connect to xyz.db
engine=create_engine('sqlite:///xyz.db')
# engine = create_engine('sqlite://///Users/EdwardArroyo/Desktop/GrEx2/xyz.db')
conn=engine.connect()

In [12]:
# Instead use sqlite3 to get a connection
#conn=sqlite3.connect('xyz.db')

In [13]:
# Use either connection with pandas DataFrame objects' to_sql method
titanic_df.to_sql('titanic',conn,index=False)
#titanic_df.to_sql('titanic',conn,index=False,if_exists='replace')

In [14]:
ls  # to check that xyz.db has been created...

GrEx 2 Progress Report 2 Examples.html
GrEx 2 Progress Report 2 Examples.ipynb
GrEx2 Progress Report 2 Examples v3.ipynb
titanic.csv
xyz.db


In [15]:
engine.table_names() # only works if you used SQLAlchemy...

['titanic']

Query the database...

In [16]:
count = pd.read_sql_query("SELECT COUNT(*) FROM titanic",conn)
count

,COUNT(*)
0,891


In [17]:
titanic_df.shape   # verify the row count with the DataFrame..

(891, 14)

In [18]:
# Reading a table from the database into a DataFrame
titanic_df2=pd.read_sql('titanic',conn)
titanic_df2.head()

,PassengerID,Survived,Pclass,Name,Sex,GCode2,GCode,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,1,1,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,2,2,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,2,2,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,2,2,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,1,1,35.0,0,0,373450,8.0500,None,S


In [19]:
titanic_df.equals(titanic_df2)

True

Finally, I want to consider a scenario that you will deal with 

In [20]:
titanic_df.columns

Index(['PassengerID', 'Survived', 'Pclass', 'Name', 'Sex', 'GCode2', 'GCode',
       'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In Part 4 (b) of GrEx2, you might want to count for each customer the number of times each product category appears in the `deptdescr` column of the **item** table. Below we consider a similar scenario: count for each age of passengers in the Titanic the number of males and females.

In [21]:
gb = titanic_df.groupby('Age').Sex.value_counts()
gb[10:20]   # here we are slicing by position

Age  Sex   
3.0  female    2
4.0  female    5
     male      5
5.0  female    4
6.0  female    2
     male      1
7.0  male      2
     female    1
8.0  female    2
     male      2
Name: Sex, dtype: int64

In [22]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html
gb.unstack(fill_value=0)[3:8]   # here we are slicing by an age range for the purposes of checking below

Sex,female,male
Age,,
3.0,2,4
4.0,5,5
5.0,4,0
6.0,2,1
7.0,1,2
8.0,2,2


Let us check the first two rows independently...

In [23]:
titanic_df[titanic_df.Age==3.0].Sex.value_counts()

male      4
female    2
Name: Sex, dtype: int64

In [24]:
titanic_df[titanic_df.Age==4.0].Sex.value_counts()

male      5
female    5
Name: Sex, dtype: int64

Still another way to do it...We select the same age range as above for the purpose of comparing values

In [40]:
gb.reset_index(name='count').pivot(index='Age', columns='Sex',values='count').fillna(0)[3:8]

Sex,female,male
Age,,
3.0,2.0,4.0
4.0,5.0,5.0
5.0,4.0,0.0
6.0,2.0,1.0
7.0,1.0,2.0
8.0,2.0,2.0


For fun, here is a "hack" using pivot_tables...We create column and count the number of rows in the column broken down by gender for each age value. This is exactly what we want...

In [26]:
titanic_AS_df = titanic_df[['Age','Sex']].reset_index('count')
titanic_AS_df.head()

,index,Age,Sex
0,0,22.0,male
1,1,38.0,female
2,2,26.0,female
3,3,35.0,female
4,4,35.0,male


In [29]:
titanic_AS_count = titanic_AS_df.pivot_table(index='Age',columns='Sex',aggfunc='count',fill_value=0)
titanic_AS_count[3:8]

index     
Sex female male
Age            
3.0      2    4
4.0      5    5
5.0      4    0
6.0      2    1
7.0      1    2
8.0      2    2

Let us convert the nonzero numbers to 1 to indicate the existance of individual of that gender.

In [39]:
titanic_AS_ind = titanic_AS_count.apply(lambda x: np.where(x>0,1,0),axis=0)
titanic_AS_ind[12:15]    # select a range of values for the purposes of comparison

index     
Sex  female male
Age             
12.0      0    1
13.0      1    0
14.0      1    1
14.5      1    0
15.0      1    1

In [37]:
titanic_AS_count[12:15]   # Looks right on the range selected..

index     
Sex  female male
Age             
12.0      0    1
13.0      2    0
14.0      4    2
14.5      1    0
15.0      4    1